# 1 Brute force ❌

First burst the first balloon, then the second one and so on. Recursively try to burst all the remaining balloons at each node in a DFS fashion. The tree has as many leaves as permutations of n, i.e. $n!$, because each path is an ordered selection of elements. There are as many nodes as k-permutations of n, $1 \leq k \leq n$, i.e $\lfloor e \cdot n! - 1 \rfloor$, see [this post](https://math.stackexchange.com/questions/2019675/sum-of-permutations-from-0-to-n). So, the brute force algorithm has a $O(n \cdot n!)$ runtime complexity, since each call costs a $O(n)$ iteration on all considered balloons (at most $n$). The callstack is at most size $n$.

In [1]:
def dfs(nums: list[int]) -> int:
    if len(nums) == 0:
        return 0
    output = 0
    for i in range(len(nums)):
        coins = (
            (nums[i - 1] if 0 <= i - 1 else 1)
            * nums[i]
            * (nums[i + 1] if i + 1 < len(nums) else 1)
        )
        balloon = nums.pop(i)
        output = max(output, coins + dfs(nums))
        nums.insert(i, balloon)
    return output

dfs([3,1,5,8])

167

# 2 Memoization of DFS backtracking ❌

In the above DFS algorithm, several paths in the recursion tree lead to identical nodes i.e. identical subproblems. For instance, with test case `[3, 1, 5, 8]`, bursting 3 then 1 vs 1 then 3 lead to the same subproblem `[5, 8]`. Why? Because the DFS algorithm tries all the permutations of the input: any path corresponds to an ordered selection of elements, i.e a permutation. Then, if two paths are a permutation of one another, they lead to the same subproblem. The set of subproblems is exactly the set of subsequences of the input, that is the number of subsets, that is $2^n$.  
  
Another way to see it: how to build a subproblem? Choose a balloon to burst, then another one, and so on... This process builds a set. The subproblem is the complement of this set.
  
Anyway, $2^n$ subproblems are still to much for an efficient dynamic solution.

# 3 What is better than subsequences?

There are much less **subarrays** than subsequences. But if one bursts `5` first, then one cannot process the remaining subarrays `[3, 1]` and `[8]` independently, because 8 has now replaced 5 as a direct neighbor of 1. On the contrary, if 5 is bursted **last**, then the neighboring subarrays can be processed independently: we know for sure their ends will never "touch", since 5 was popped after all balloons of both subarrays.

Recursive logic on a given subarray with bounds indexed `l` and `r`. Try bursting all the balloons between `l` and `r` included. Each considered balloon would be the last to be bursted in the array. In this case, their neighbors would be balloons at indices `l - 1` and `r + 1`.

## 3.1 Reversed logic: backtracking with subarrays ❌

Since we are still trying all possible ordered choices of balloons, the time complexity remains $O(n \cdot n!)$ (see section 1).  
Regarding memory, the callstack is at most of size $n$ as well. Even if that does not change the space complexity, there is also the insertion of the value `1` at the beginning of nums. Still, the space complexity is $O(n)$ overall.

In [6]:
nums = [3, 1, 5, 8]
nums.insert(0, 1)
nums.append(1)

def dfs(l: int, r: int) -> int:
    if l > r:
        return 0
    output = 0
    for i in range(l, r + 1):
        coins = nums[l - 1] * nums[i] * nums[r + 1]
        gain = coins + dfs(l, i - 1) + dfs(i + 1, r)
        output = max(output, gain)
    return output

dfs(1, len(nums) - 2)

167

## 3.2 Memoized backtracking (top-down) with subarrays ✅ 

Now the time complexity is finally polynomial: $n^2$ states (= subarrays) $\times$ an iteration on at most $n$ balloons is $O(n^3)$.  
The space complexity is $O(n^2)$ because the results for all possible subarrays are stored in the cache. This quadratic space 'eats' the linear space of the callstack + insertion at index 0 of `nums`.

In [8]:
nums = [3, 1, 5, 8]
nums.insert(0, 1)
nums.append(1)
cache = {}

def dfs(l: int, r: int) -> int:
    if l > r:
        return 0
    output = 0
    for i in range(l, r + 1):
        coins = nums[l - 1] * nums[i] * nums[r + 1]
        if (l, i - 1) not in cache:
            cache[(l, i - 1)] = dfs(l, i - 1)
        if (i + 1, r) not in cache:
            cache[(i + 1, r)] = dfs(i + 1, r)
        gain = coins + cache[(l, i - 1)] + cache[(i + 1, r)]
        output = max(output, gain)
    return output

dfs(1, len(nums) - 2)

167

## 3.2 Dynamic programming (bottom-up) with subarrays ✅

Still $O(n^3)$ for time and $O(n^2)$ for space complexity.

In [12]:
nums = [3, 1, 5, 8]
nums.insert(0, 1)
nums.append(1)

dp = [[0] * len(nums) for _ in range(len(nums))]
for i in range(1, len(nums) - 1):
    dp[i][i] = nums[i - 1] * nums[i] * nums[i + 1]
for sublength in range(1, (len(nums) - 2) + 1): #len(nums) - 2 is the 'real' length of nums
    for l in range(1, (len(nums) - 2) - sublength + 2):
        r = l + sublength - 1
        for i in range(l, r + 1):
            gain = nums[l - 1] * nums[i] * nums[r + 1]
            remaining = dp[l][i - 1] + dp[i + 1][r]
            dp[l][r] = max(dp[l][r], gain + remaining)
dp[1][len(nums) - 2]

167